# Indice

[1.1. Creating the spark session and context](#1.1.)

### 1.1. Creating the spark session and context [Indice](#Indice)

In [3]:
# Import the PySpark module
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

In [4]:
##Create SparkContext
sc = SparkContext.getOrCreate()

# Create SparkSession object
spark = SparkSession.builder \
                    .master('local[*]') \
                    .appName('test') \
                    .getOrCreate()

In [5]:
# What version of Spark?
print(spark.version)

2.4.4


### Read data from CSV file (infering schema) [Indice](#Indice)

In [6]:
## Is not the best choose for large data sets
flights = spark.read.csv('flights.csv',
                         sep=',',
                         header=True,
                         inferSchema=True,
                         nullValue='NA'
                        )

### Read data from CSV file (defining schema)

In [8]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

# Specify column names and types
schema = StructType([
    StructField("id", IntegerType()),
    StructField("text", StringType()),
    StructField("label", IntegerType())
])

# Load data from a delimited file
sms = spark.read.csv('sms.csv', sep=';', header=False, schema=schema)

# Print schema of DataFrame
sms.printSchema()

root
 |-- id: integer (nullable = true)
 |-- text: string (nullable = true)
 |-- label: integer (nullable = true)



### Exploring data

In [9]:
# Get number of records
print("The data contain %d records." % flights.count())

The data contain 10000 records.


In [11]:
# View the first five records
flights.show(5)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|   7|    54|
+----+-----+---+--------+---------+-----

In [12]:
flights.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
year,10000,2014.0,0.0,2014,2014
month,10000,6.6438,3.3191600205962097,1,12
day,10000,15.7009,8.895142019392079,1,31
dep_time,9952,1277.1158561093248,524.1142951055886,1,2400
dep_delay,9952,6.068629421221865,28.808608062751805,-19,886
arr_time,9945,1477.7236802413272,526.5936522261665,1,2400
arr_delay,9925,2.2530982367758186,31.074918600451877,-58,900
carrier,10000,None,None,AA,WN
tailnum,9986,None,None,D942DN,N988CA


In [13]:
# Check column data types
flights.dtypes

[('year', 'int'),
 ('month', 'int'),
 ('day', 'int'),
 ('dep_time', 'int'),
 ('dep_delay', 'int'),
 ('arr_time', 'int'),
 ('arr_delay', 'int'),
 ('carrier', 'string'),
 ('tailnum', 'string'),
 ('flight', 'int'),
 ('origin', 'string'),
 ('dest', 'string'),
 ('air_time', 'int'),
 ('distance', 'int'),
 ('hour', 'int'),
 ('minute', 'int')]

### Selecting columns

In [14]:
flights_drop = flights.drop('distance','hour')

In [15]:
flights_select = flights.select('year','month')

### Filtering out missing data

In [16]:
flights.filter('arr_delay is null').count()

75

In [17]:
flights_no_miss = flights.filter('arr_delay is null')

In [18]:
flights_no_miss_2 = flights.dropna()

In [ ]:
# Terminate the cluster
spark.stop()

### Column manipulation

In [ ]:
# Import the required function
from pyspark.sql.functions import round

# Convert 'mile' to 'km' and drop 'mile' column
flights_km = flights.withColumn('km', round(flights.mile * 1.60934, 0)) \
                    .drop('mile')
flights_km.head()
# Create 'label' column indicating whether flight delayed (1) or not (0)
flights_km = flights_km.withColumn('label', (flights_km.delay >=15).cast('integer'))

# Check first five records
flights_km.show(5)

### Indexing categorical data

In [ ]:
from pyspark.ml.feature import StringIndexer

# Create an indexer
indexer = StringIndexer(inputCol='carrier', outputCol='carrier_idx')

# Indexer identifies categories in the data
indexer_model = indexer.fit(flights)

# Indexer creates a new column with numeric index values
flights_indexed = indexer_model.transform(flights)

# Repeat the process for the other categorical feature
flights_indexed = StringIndexer(inputCol='org', outputCol='org_idx').fit(flights_indexed).transform(flights_indexed)

### Assembling columns

In [ ]:
# Create an assembler object

assembler = VectorAssembler(
    inputCols=['mon', 
               'dom' , 
               'dow',
               'carrier_idx',
               'org_idx',
               'km',
               'depart',
               'duration'
              ], outputCol='features')

# Consolidate predictor columns
flights_assembled = assembler.transform(flights)

### Train/test split

In [ ]:
# Split into training and testing sets in a 80:20 ratio
flights_train, flights_test = flights.randomSplit([0.8, 0.2], seed=17)

# Check that training set has around 80% of records
training_ratio = flights_train.count() / flights.count()
print(training_ratio)

### Build a Decision Tree

In [ ]:
# Import the Decision Tree Classifier class
from pyspark.ml.classification import DecisionTreeClassifier

# Create a classifier object and fit to the training data
tree = DecisionTreeClassifier()
tree_model = tree.fit(flights_train)

# Create predictions for the testing data and take a look at the predictions
prediction = tree_model.transform(flights_test)
prediction.select('label', 'prediction', 'probability').show(5, False)

### Evaluate the Decision Tree

In [ ]:
# Create a confusion matrix
prediction.groupBy('label', 'prediction').count().show()

# Calculate the elements of the confusion matrix
TN = prediction.filter('prediction = 0 AND label = prediction').count()
TP = prediction.filter('prediction = 1 AND label = prediction').count()
FN = prediction.filter('prediction = 0 AND label != prediction').count()
FP = prediction.filter('prediction = 1 AND label != prediction').count()

# Accuracy measures the proportion of correct predictions
accuracy = (TN + TP) / (TN + TP + FN + FP) 
print(accuracy)